In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import math

from wordcloud import WordCloud

In [ ]:
# generate wordcloud
def generate_wordcloud(word_count):
    wc = WordCloud(width = 3000, height = 3000,
                    background_color ='white',
                    min_font_size = 10, max_words=100).generate_from_frequencies(word_count)

    plt.imshow(wc)
    plt.savefig("output/Keyword_Frequence_Wordcloud.jpg")

In [ ]:
def normalize_c_tf_idf(word, cat_freqs, doc_freqs):
    cat_word_count = {c:sum(word_count_category[c].values()) for c in word_count_category}
    A = sum(cat_word_count.values())/len(cat_word_count)

    norm_c_tf = cat_freqs[word] / sum(cat_freqs.values())
    # idf = sum(doc_freqs.values()) / doc_freqs[word]
    idf = 1 + A / doc_freqs[word]
    return norm_c_tf * math.log(idf)    

In [ ]:
def create_keyword_ctfidf(word_count_category, word_count, top_n = 5, min_occurance = 0):
    res_df = pd.DataFrame()
    for cat in word_count_category:
        ctfid_scores = [""]*top_n
        
        sorted_cat = dict(sorted(word_count_category[cat].items(), key=lambda item: item[1], reverse=True))
        # ratios = {word:normalized_c_tf_idf(word,word_count_category[cat],doc_count) for word in sorted_cat  if sorted_cat[word] > min_occurance}
        ratios = {word:normalize_c_tf_idf(word,word_count_category[cat],word_count) for word in sorted_cat  if sorted_cat[word] > min_occurance}
        sorted_ratios = dict(sorted(ratios.items(), key=lambda item: item[1], reverse=True))
        
        for i in range(min(top_n, len(sorted_ratios))):
            word = list(sorted_ratios.keys())[i]
            ctfid_scores[i] = f"{word}: {round(sorted_ratios[word],3)}"
        res_df.insert(len(res_df.columns),cat, ctfid_scores, True)
        
    return res_df